# Data Gathering and Management Lab - 094290
## Lab 1 - ETL and EDA
#### Nitzan Shamir 206348187 & Omer Shubi 312236219

In this part of the project we have three main tasks.

The first is to **extract** the data from the given database.

Then, we perform explanatory data analysis (**EDA**), derive meaningful insights and implement the necessary data **transformations**.

Finally, we **load** the data to our selected datawarehouse that runs on our Virtual Machine (VM).


Note, we hide the code cells so that the notebook stays clean. The full code is in the included zip Folder.

In [3]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SQLContext

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.elasticsearch:elasticsearch-hadoop:7.9.3 pyspark-shell'

!python -m pip install elasticsearch
from elasticsearch import Elasticsearch

from pyspark.sql.functions import date_format

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd # for small table fancy display only

# Extract

In this phase we read the data from a given database. 

We start by extracting the data from */mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile*.

The data itself contains bus telemetry of Dublin public transportation (total of ~170GB in JSON format).

Initially we loaded the data without specifying the schema, and after the first time we store here the schema of the data for faster loading.

In [5]:
full_schema = StructType([StructField('_id',StructType([StructField('$oid',StringType(),True)]),True),
                     StructField('actualDelay',LongType(),True),
                     StructField('angle',DoubleType(),True),
                     StructField('anomaly',BooleanType(),True),
                     StructField('areaId',LongType(),True),
                     StructField('areaId1',LongType(),True),
                     StructField('areaId2',LongType(),True),
                     StructField('areaId3',LongType(),True),
                     StructField('atStop',BooleanType(),True),
                     StructField('busStop',LongType(),True),
                     StructField('calendar',StructType([StructField('$numberLong',StringType(),True)]),True),
                     StructField('congestion',BooleanType(),True),
                     StructField('currentHour',LongType(),True),
                     StructField('dateType',LongType(),True),
                     StructField('dateTypeEnum',StringType(),True),
                     StructField('delay',LongType(),True),
                     StructField('direction',LongType(),True),
                     StructField('distanceCovered',DoubleType(),True),
                     StructField('ellapsedTime',LongType(),True),
                     StructField('filteredActualDelay',LongType(),True),
                     StructField('gridID',StringType(),True),
                     StructField('journeyPatternId',StringType(),True),
                     StructField('justLeftStop',BooleanType(),True),
                     StructField('justStopped',BooleanType(),True),
                     StructField('latitude',DoubleType(),True),
                     StructField('lineId',StringType(),True),
                     StructField('loc',StructType([StructField('coordinates',ArrayType(DoubleType(),True),True),StructField('type',StringType(),True)]),True),
                     StructField('longitude',DoubleType(),True),
                     StructField('poiId',LongType(),True),
                     StructField('poiId2',LongType(),True),
                     StructField('probability',DoubleType(),True),
                     StructField('systemTimestamp',DoubleType(),True),
                     StructField('timestamp',StructType([StructField('$numberLong',StringType(),True)]),True),
                     StructField('vehicleId',LongType(),True),
                     StructField('vehicleSpeed',LongType(),True)])

In [6]:
%fs ls /mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile

path,name,size
dbfs:/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile,busFile,179210947520


In [7]:
df = spark.read.json('/mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile', schema=full_schema)
# df = spark.createDataFrame(df.take(1000)).cache()
# df = df.sample(False, 0.001, seed=0).cache()

# EDA

In this phase we explore the data.

First, we perform a basic analysis of the raw data.

Then, based on the first part we perform basic transformation of data to allow us to continue the analysis.

Finally, we perform in-depth analysis and provide meaningful insights.

## Basic analysis

We display the raw data, the data type of each feature and summary statistics for the different features (dividing by numeric/categorical/boolean).

Note: we refer to the types of the features after quik exploration, and not specifically for the type defined in the dataframe. For example, the type of lineId' feature is *long* but it is likely to assume that it's a categorial feature and not numeric feature.

In [10]:
display(df)

_id,actualDelay,angle,anomaly,areaId,areaId1,areaId2,areaId3,atStop,busStop,calendar,congestion,currentHour,dateType,dateTypeEnum,delay,direction,distanceCovered,ellapsedTime,filteredActualDelay,gridID,journeyPatternId,justLeftStop,justStopped,latitude,lineId,loc,longitude,poiId,poiId2,probability,systemTimestamp,timestamp,vehicleId,vehicleSpeed
List(595a2300e45b4b2ea81ae778),0,45.0,false,15001,14,234,3750,false,2377,List(1499079397000000),false,23,1,WEEKEND,-2,0,0.22027568625824304,21000,0,"132,86",00271001,false,false,53.315958,27,"List(List(-6.334248, 53.315958), Point)",-6.334248,0,0,0.0,6.15949535E8,List(1499079397000),44087,0
List(595a2300e45b4b2ea81ae779),-25,225.0,false,15138,14,236,3784,false,2485,List(1499079397000000),false,23,1,WEEKEND,-104,0,0.16586637987086802,18000,0,"146,88",00181001,false,false,53.319117,18,"List(List(-6.290017, 53.319117), Point)",-6.290017,0,0,0.0,6.15949538E8,List(1499079397000),33467,0
List(595a2300e45b4b2ea81ae77a),0,225.0,false,18342,17,286,4585,false,113,List(1499079397000000),false,23,1,WEEKEND,124,0,0.048167532049173775,23000,0,"155,131",00130002,true,false,53.395714,13,"List(List(-6.26408, 53.395714), Point)",-6.26408,0,0,0.0,6.1594954E8,List(1499079397000),43041,0
List(595a2300e45b4b2ea81ae77b),0,225.0,false,3062,11,191,3062,false,7239,List(1499079397000000),false,23,1,WEEKEND,-147,0,0.23610840881390818,24000,0,"120,107",025B0002,false,false,53.353845,25,"List(List(-6.367692, 53.353845), Point)",-6.367692,0,0,0.0,6.15949542E8,List(1499079397000),33401,0
List(595a2300e45b4b2ea81ae77c),0,45.0,false,15536,14,242,3883,false,779,List(1499079397000000),false,23,1,WEEKEND,-423,0,0.0758441143128766,24000,0,"160,93",00180001,true,false,53.327876,18,"List(List(-6.248047, 53.327876), Point)",-6.248047,0,0,0.0,6.15949544E8,List(1499079397000),33464,0
List(595a2300e45b4b2ea81ae77d),-22,45.0,false,73060,17,285,4565,false,404,List(1499079397000000),false,23,1,WEEKEND,61,0,0.08946480481101295,22000,0,"157,103",00110001,false,false,53.345593,11,"List(List(-6.257684, 53.345593), Point)",-6.257684,0,0,0.0,6.15949546E8,List(1499079397000),44110,0
List(595a2300e45b4b2ea81ae77e),0,45.0,false,73060,17,285,4565,false,1278,List(1499079397000000),false,23,1,WEEKEND,194,0,0.03299085710397742,24000,0,"157,103",01231001,true,false,53.346069,123,"List(List(-6.257612, 53.346069), Point)",-6.257612,0,0,0.0,6.15949547E8,List(1499079397000),44103,0
List(595a2300e45b4b2ea81ae77f),-25,45.0,false,15646,14,244,3911,false,2007,List(1499079397000000),false,23,1,WEEKEND,-373,0,0.448449642683063,30000,0,"170,83",046A0001,false,false,53.310553,46A,"List(List(-6.21956, 53.310553), Point)",-6.21956,0,0,0.0,6.15949549E8,List(1499079397000),36013,0
List(595a2300e45b4b2ea81ae780),0,45.0,false,1164,17,290,1164,false,4906,List(1499079397000000),false,23,1,WEEKEND,12,0,0.1615753977654222,22000,0,"162,166",00411001,false,false,53.459475,41,"List(List(-6.242159, 53.459475), Point)",-6.242159,0,0,0.0,6.15949551E8,List(1499079397000),33336,0
List(595a2300e45b4b2ea81ae781),20,-2.0,false,18309,17,285,4577,true,510,List(1499079397000000),false,23,1,WEEKEND,87,0,0.0,19000,0,"161,110",01231001,false,false,53.359226,123,"List(List(-6.245823, 53.359226), Point)",-6.245823,0,0,0.0,6.15949553E8,List(1499079397000),44107,0


In [11]:
df.count()

Out[41]: 237143372

In [12]:
display(spark.createDataFrame(df.dtypes).withColumnRenamed('_1', 'feature').withColumnRenamed('_2', 'feature_type'))

feature,feature_type
_id,struct<$oid:string>
actualDelay,bigint
angle,double
anomaly,boolean
areaId,bigint
areaId1,bigint
areaId2,bigint
areaId3,bigint
atStop,boolean
busStop,bigint


In [13]:
numerical_columns = ["summary", "actualDelay", "angle", "delay", "distanceCovered", "ellapsedTime", "filteredActualDelay", "latitude", "longitude", "vehicleSpeed", "probability"]

display(df.describe().select(numerical_columns))

summary,actualDelay,angle,delay,distanceCovered,ellapsedTime,filteredActualDelay,latitude,longitude,vehicleSpeed,probability
count,237143372,237143372,237143372,237143372,237143372,237143372,237143372,237143372,237143372,237143372
mean,0.26890048607388445,111.04436103742339,187.79413873308675,0.11713560517908854,36656.47034402462,-0.05711734587294306,53.34313177173547,-6.269471414619056,49686.90628565406,0.0
stddev,62.919876696499166,96.55944488407489,420.2325261387324,0.2821353469965412,1077726.7729727218,65.60073695995933,0.05297648446116861,0.07864545160793346,1.0327796235956786E7,0.0
min,-227015,-2.0,-30404,0.0,-140000,-140000,53.068017,-6.617517,-3836,0.0
max,117183,225.0,226954,41.46665813130014,1066270000,0,53.608717,-6.052983,2147483647,0.0


In [14]:
categorial_columns = ["vehicleId", "areaId", "areaId1", "areaId2", "areaId3", "busStop", "gridID", "journeyPatternId","currentHour",  "dateType", "dateTypeEnum", "lineId", "poiId", "poiId2","direction", "anomaly"]
cat_df = pd.DataFrame(df.agg(*(F.countDistinct(F.col(c)).alias("Unique Count of "+c) for c in categorial_columns)).collect(), index=['Number of Unique Categories'],columns=categorial_columns)
cat_df.transpose()

,Number of Unique Categories
vehicleId,1165
areaId,885
areaId1,13
areaId2,124
areaId3,579
busStop,4774
gridID,7995
journeyPatternId,584
currentHour,24
dateType,2


In [15]:
bool_cols= ["justLeftStop","justStopped", "congestion","atStop"]

raw_bool_df = df.groupby(bool_cols).count()
bool_df = pd.DataFrame()

for col in bool_cols:
    bool_df[col] = pd.DataFrame(raw_bool_df.groupby(col).sum().collect(), index=['Count True', 'Count False'],columns=['other1', col])[col]
bool_df.transpose()

,Count True,Count False
justLeftStop,26383425,210759947
justStopped,26311872,210831500
congestion,2058641,235084731
atStop,50660852,186482520


## Basic transformations

To be able to continue to analyze the data, we perform basic transformations on the data into the relevant types. 

We parse the relevant variables to the right format - including flatten the nested objects and transform timestamp variables from unix time to timestamp format: yyyy-MM-dd HH:mm:ss.

We also drop the id column as the spark datafram has a built-in index and we have no need of another one.

In [17]:
df = df.withColumn('timestamp', df['timestamp']['$numberLong'].cast('bigint'))\
       .withColumn('calendar', df['calendar']['$numberLong'].cast('bigint'))

df = df.withColumn('date', F.from_unixtime((df['timestamp'])/1000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
       .withColumn('loc', df['loc']['coordinates'])\
       .drop("timestamp")\
       .withColumn('calendar', F.from_unixtime((df['calendar'])/1000000, 'yyyy-MM-dd HH:mm:ss').cast('timestamp'))\
       .withColumn('systemTimestamp', F.to_timestamp(df['systemTimestamp']))

# drop id column
df =  df.drop("_id")

In [18]:
display(df)

actualDelay,angle,anomaly,areaId,areaId1,areaId2,areaId3,atStop,busStop,calendar,congestion,currentHour,dateType,dateTypeEnum,delay,direction,distanceCovered,ellapsedTime,filteredActualDelay,gridID,journeyPatternId,justLeftStop,justStopped,latitude,lineId,loc,longitude,poiId,poiId2,probability,systemTimestamp,vehicleId,vehicleSpeed,date
0,45.0,false,15001,14,234,3750,false,2377,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,-2,0,0.22027568625824304,21000,0,"132,86",00271001,false,false,53.315958,27,"List(-6.334248, 53.315958)",-6.334248,0,0,0.0,1989-07-09T01:05:35.000+0000,44087,0,2017-07-03T10:56:37.000+0000
-25,225.0,false,15138,14,236,3784,false,2485,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,-104,0,0.16586637987086802,18000,0,"146,88",00181001,false,false,53.319117,18,"List(-6.290017, 53.319117)",-6.290017,0,0,0.0,1989-07-09T01:05:38.000+0000,33467,0,2017-07-03T10:56:37.000+0000
0,225.0,false,18342,17,286,4585,false,113,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,124,0,0.048167532049173775,23000,0,"155,131",00130002,true,false,53.395714,13,"List(-6.26408, 53.395714)",-6.26408,0,0,0.0,1989-07-09T01:05:40.000+0000,43041,0,2017-07-03T10:56:37.000+0000
0,225.0,false,3062,11,191,3062,false,7239,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,-147,0,0.23610840881390818,24000,0,"120,107",025B0002,false,false,53.353845,25,"List(-6.367692, 53.353845)",-6.367692,0,0,0.0,1989-07-09T01:05:42.000+0000,33401,0,2017-07-03T10:56:37.000+0000
0,45.0,false,15536,14,242,3883,false,779,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,-423,0,0.0758441143128766,24000,0,"160,93",00180001,true,false,53.327876,18,"List(-6.248047, 53.327876)",-6.248047,0,0,0.0,1989-07-09T01:05:44.000+0000,33464,0,2017-07-03T10:56:37.000+0000
-22,45.0,false,73060,17,285,4565,false,404,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,61,0,0.08946480481101295,22000,0,"157,103",00110001,false,false,53.345593,11,"List(-6.257684, 53.345593)",-6.257684,0,0,0.0,1989-07-09T01:05:46.000+0000,44110,0,2017-07-03T10:56:37.000+0000
0,45.0,false,73060,17,285,4565,false,1278,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,194,0,0.03299085710397742,24000,0,"157,103",01231001,true,false,53.346069,123,"List(-6.257612, 53.346069)",-6.257612,0,0,0.0,1989-07-09T01:05:47.000+0000,44103,0,2017-07-03T10:56:37.000+0000
-25,45.0,false,15646,14,244,3911,false,2007,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,-373,0,0.448449642683063,30000,0,"170,83",046A0001,false,false,53.310553,46A,"List(-6.21956, 53.310553)",-6.21956,0,0,0.0,1989-07-09T01:05:49.000+0000,36013,0,2017-07-03T10:56:37.000+0000
0,45.0,false,1164,17,290,1164,false,4906,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,12,0,0.1615753977654222,22000,0,"162,166",00411001,false,false,53.459475,41,"List(-6.242159, 53.459475)",-6.242159,0,0,0.0,1989-07-09T01:05:51.000+0000,33336,0,2017-07-03T10:56:37.000+0000
20,-2.0,false,18309,17,285,4577,true,510,2017-07-03T10:56:37.000+0000,false,23,1,WEEKEND,87,0,0.0,19000,0,"161,110",01231001,false,false,53.359226,123,"List(-6.245823, 53.359226)",-6.245823,0,0,0.0,1989-07-09T01:05:53.000+0000,44107,0,2017-07-03T10:56:37.000+0000


In [19]:
display(df.select(F.min("date"), F.max("date")))

min(date),max(date)
2017-07-03T10:56:37.000+0000,2018-09-11T18:10:43.000+0000


## Data information summary

The data consists of ~230 million records of 35 features that are of types - boolean, long, float, integers, string, arrays and nested objects with more fields structured inside.

The data was collected between July 2017 and August 2018. 

We understand that each row in the data represents a log entry of a single bus at a given time.

It includes information about the
- time
- location (loc, areaIDs, busStop)
- traffic condition (delays, congestion)
- bus status (vehicleId, AtStop, JustStopped, vehicleSpeed)
- Journey Status (distanceCovered, ellapsedTime, LineId)

and more.

From the above we see for example that there are 1165 unique buses in the dataset, 68 different lines, and we see the imbalance in the binary features.

Additionally, taking a quick look at the tables above, we see that there are many features that are unclear, appear uninformative or even wrong. 

The data is acquired by physical sensors (such as gps sensor) located on buses. As such, missing and inaccurate measurements (e.g. gps location and speed) are something we can expect to see.

## In-depth analysis

Following the basic analysis, in this stage, we try to extract and understand more information about the data. 

We do so by running different queries and by visual examination of the different features and the relations between them.

Specifically, we check for all-zero features, features that have the same values (redundant), features that have uncertainties (inconsistent)  and that do not agree with each other.

Additionally we try to figure out features that are 'useless', even though we don't have a specific goal yet.

Finally, for the features that seem relevant to us, we go out to figure the units and the relationships between them.

By exploring the data, we find interesting insights, which we describe below, that allow us at the end to determine our datawarehouse provider and diagram.

### Feature analysis

In [23]:
summary_df = df.where((df["probability"] != 0) | (df["poiId"] != 0) | (df["poiId2"] != 0) | (df["filteredActualDelay"] != 0) | (df["anomaly"] != False)).cache()

In [24]:
# explore probability column
print("number of rows where probability != 0 :", summary_df.where(summary_df["probability"] != 0).count())
# explore poiId column
print("number of rows where poiId != 0 :", summary_df.where(summary_df["poiId"] != 0).count())
# explore poiId2 column
print("number of rows where poiId2 != 0 :", summary_df.where(summary_df["poiId2"] != 0).count())
# explore filteredActualDelay column
print("number of rows where filteredActualDelay != 0 :", summary_df.where(summary_df["filteredActualDelay"] != 0).count())
# explore anomaly column
print("number of rows where anomaly != False: ", summary_df.where(summary_df["anomaly"] != False).count())

number of rows where probability != 0 : 0
number of rows where poiId != 0 : 0
number of rows where poiId2 != 0 : 0
number of rows where filteredActualDelay != 0 : 905
number of rows where anomaly != False: 0

In [25]:
# explore calendar date
display(df.select('calendar','date'))

calendar,date
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000
2017-07-03T10:56:37.000+0000,2017-07-03T10:56:37.000+0000


In [26]:
# explore loc, longitude, latitude columns
display(df.select("loc", "longitude", "latitude"))

loc,longitude,latitude
"List(-6.334248, 53.315958)",-6.334248,53.315958
"List(-6.290017, 53.319117)",-6.290017,53.319117
"List(-6.26408, 53.395714)",-6.26408,53.395714
"List(-6.367692, 53.353845)",-6.367692,53.353845
"List(-6.248047, 53.327876)",-6.248047,53.327876
"List(-6.257684, 53.345593)",-6.257684,53.345593
"List(-6.257612, 53.346069)",-6.257612,53.346069
"List(-6.21956, 53.310553)",-6.21956,53.310553
"List(-6.242159, 53.459475)",-6.242159,53.459475
"List(-6.245823, 53.359226)",-6.245823,53.359226


In [27]:
# explore dateTypeEnum, dateType, is_weekend columns
display(df.withColumn("is_weekend", date_format("date", 'EEE').isin(["Sat", "Sun"]).cast("int")).select('is_weekend', 'dateType', 'dateTypeEnum'))

is_weekend,dateType,dateTypeEnum
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY
0,0,WEEKDAY


In [28]:
# explore dateTime, currentHour columns
display(df.select("date", "currentHour"))

date,currentHour
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23
2017-07-03T10:56:37.000+0000,23


### Interim conclusions

- We found that the features: "probability", "poiId", "poiId2", "anomaly" contain only zeros (/False) entries. And the feature "filteredActualDelay" contains almost only zero entries (~99.9997% of data)

- We found that there is redundancy in the data. The features "calendar" and "date" have the same values and the feature "loc" is an array consisting of the features "longitude" and "latitude".

- We found that there is uncertainty in the data. The feature "currentHour" does not match the hour described in "date". Similarly, "dateTypeEnum" and "dateType" are not consistent with the day part of the "date".

### Our insights

In order to analyze the data, we query the data in databricks and also use Kibana UI for visualizations.

In order to do so, we first upload a sample of the data to the VM (ElasticSearch + Kibana).

Note: Reasons why we choose Elasticsearch as our data warehouse appear in the Load section.

In addition to DataBricks, we use kibana for our explorations and visualisations as the core feature of Kibana is data querying & analysis, and it allows us to see complex relations that otherwise would have been hard to see.

Specifically, Kibana’s visualization features allow us to visualize data in alternate ways using geospatial maps, overlayed and displayed differently according to our needs.

Additionally it is easy to create heat maps, line graphs, histograms, pie charts and more.

#### #1 insight - lineId, journeyPatternId relationship

We aim to understand the relationship between "lineId" and "journeyPatternId".

First, we group the data by lineId and journeyPatternId.

Then we count how many lineId there are for each journeyPatternId and how many journeyPatternId there are for each lineId.

We see that each journeyPattern has one line, and that each line has multiple journeyPatterns. (the first two cells below)

To further understand the meaning of these features, we examine a specific lineId (lineId == 1) and look at its' journeyPatternIds. (the third & fourth cells below)

From the numerical data it is hard to understand what each journeyPatternId represents, so we look at the geospatial information of each record of each journeyPatternId for lineId 1.

We can see from the map that each journeyPatternId corresponds to different routes of a specific line number.

Specifically, as there are north->south and south->north variants of the line, there are 2 journey patterns.

We note that these 'trip patterns' can be on the same road, or on different roads (for example if the road is a one-way street). 

Additionally, from further exploration of lines with multiple journey patterns we understand that some lines go through different routes or end at different locations based on other factors.
Think of Line 299 from M'rar to Haifa University. During weekdays it passes through the Technion. However, during weekends when the Technion is closed, it does a detour around the peripheral road.

In [32]:
display(df.groupby("lineId", "journeyPatternId").count().groupby("journeyPatternId").count().withColumnRenamed('count', 'Number of Unique LineIds'))

journeyPatternId,Number of Unique LineIds
033B0001,1
025A1002,1
00271001,1
041C1001,1
01511001,1
00760001,1
041B1002,1
079A1001,1
01040002,1
07570002,1


In [33]:
display(df.groupby("lineId", "journeyPatternId").count().groupby("lineId").count().withColumnRenamed('count', 'Number of Unique journeyPatternId'))

lineId,Number of Unique journeyPatternId
17A,2
7,4
747,2
15,6
11,2
42,4
59,2
16,2
29A,2
31,6


In [34]:
# select data for graph
display(df.where(F.col("lineId") == '1').select("journeyPatternId", "loc"))

journeyPatternId,loc
00010001,"List(-6.26219, 53.391176)"
00011001,"List(-6.248621, 53.343996)"
00010001,"List(-6.25909, 53.362136)"
00010001,"List(-6.233049, 53.341719)"
00010001,"List(-6.26219, 53.391176)"
00011001,"List(-6.249921, 53.344243)"
00010001,"List(-6.25972, 53.361407)"
00011001,"List(-6.254699, 53.369222)"
00011001,"List(-6.251863, 53.344656)"
00010001,"List(-6.260836, 53.360215)"


In [35]:
img = mpimg.imread("/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/ine_and_journeypattern.png")
plt.figure(figsize = (16,16))
imgplot = plt.imshow(img, interpolation='bilinear')
plt.axis('off')
plt.show()

In the image above we plot the coordinates of each JourneyPatternId for lineId 1, each in different color.

#### #2 Insight - areaId, areaId, areaId1, areaId2, areaId3 relationship

We aim to understand the relationship between the different areaIds.

From the queries (displayed below) we see that each areaId corresponds to exactly one areaId3. 

Similarly, each areaId3 corresponds to exactly one areaId2. And each areaId2 corresponds to exactly one areaId1.

Therefore, we conclude that each area feature represents a more specific partition of the map. 

To verify this we also plot the records on the geospatial map and look at the areaIds as well. The Map and a detailed description can be seen below. As expected we see a complete match.

Note: gridId seems to be a roughly more specific partition of areaId, but there isn't a full match between the sections.
The resolution of areaId seems enough, so we leave gridId aside.

In [38]:
display(df.groupby('areaId', 'areaId3').count().groupby('areaId').count().withColumnRenamed('count', 'Number of Unique areaId3'))

areaId,Number of Unique areaId3
18295,1
964,1
3764,1
15173,1
2941,1
558,1
1010,1
17135,1
17757,1
2909,1


In [39]:
display(df.groupby('areaId3', 'areaId2').count().groupby('areaId3').count().withColumnRenamed('count', 'Number of Unique areaId2'))

areaId3,Number of Unique areaId2
964,1
4590,1
3764,1
3061,1
2941,1
558,1
1010,1
2909,1
270,1
3034,1


In [40]:
display(df.groupby('areaId2', 'areaId1').count().groupby('areaId2').count().withColumnRenamed('count', 'Number of Unique areaId1'))

areaId2,Number of Unique areaId1
191,1
270,1
278,1
243,1
277,1
287,1
241,1
330,1
184,1
325,1


In [41]:
display(df.groupby('gridID', 'areaId').count().groupby('gridID').count().withColumnRenamed('count', 'Number of Unique areaId3'))

gridID,Number of Unique areaId3
"163,113",2
"152,106",3
"157,101",3
"122,107",1
"201,152",1
"182,78",1
"158,96",2
"185,197",1
"171,52",2
"161,132",3


In [42]:
img = mpimg.imread("/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/areaIds.png")
plt.figure(figsize = (25,25))
imgplot = plt.imshow(img, interpolation='bilinear')
plt.axis('off')
plt.show()

In the image above we plot the same records 4 times.

In each section the records are plotted in a color corresponding to value of the field in the title.

For example In 'AreaId3' the blue points are of a specfic areaId3 (4570) and the brown are of a different areaId3 (4483).

#### #3 insight - distanceCovered,  ellapsedTime,  vehicleSpeed units

For numerical features, the raw data provides us only numerical data. For some features it is not clear what they represent and in what units they are measured.

Therefore, we aim to understand the meaning and measurement units of the numerical columns "distanceCovered", "ellapsedTime" and "vehicleSpeed" - 3 related features that provide information about buses during their trips.

The first step is to understand the features' meaning and what they represent exactly. 

For example, it is reasonable to assume that distanceCovered is some distance measured along the route/drive, but is it measured from the beggining of the route or between each two sequential records?

After understanding that journeyPatternId is a route of a specific line bus number, we explore these features, looking at a specific segment of a specfic trip.

We explore sequential records (records from a single bus that were recorded one after the other, an example can be seen below).
From the raw data and the map view, we see that elapsed time and distance covered are calculated relatively to the previous record. This is as they do not accumulate and they match the differential values. Meaning they represent how much distance\time has passed since the previous log record of the same bus.

The second step is to understand the units of these features. 

We do so by exploring adjacent coordinates.

We can see from the map and the numerical data that:

For 'ellapsedTime' it is easy to see that it is measured in milliseconds, as for example we have two records that are 20 seconds between them and ellapsedTime shows 20000.

For 'distanceCovered', we use gps location calculator to calculate the distance between the latitude and longitude coordinates of sequential records. We get that the units of distanceCovered are in Kilometers.

After exploring ellapsedTime and distanceCovered we manually compute the average segment speed by distanceCovered/(ellapsedTime/3,600,000) and get that for records that have the vehicleSpeed filled in there is an exact match. This means that vehicleSpeed is the average segment speed, in Kph, between two sequential records.

Note: we see that there records with unreasonable speed (negative/extremely high values), and records where the speed is zero even when it seems the bus is moving. We hope to address this in the next parts of the project.

In [45]:
img = mpimg.imread('/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/distances_and_time_and_speed-1.png')
plt.figure(figsize = (20,20))
imgplot = plt.imshow(img, interpolation='bilinear')
plt.axis('off')

plt.show()

In the image above we plot adjacent coordinates' details.

In [47]:
img = mpimg.imread("/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/gps_coordinates-1.png")
plt.figure(figsize = (10,10))
imgplot = plt.imshow(img, interpolation='bilinear')
plt.axis('off')

plt.show()

In the image above we plot the caculatation of the gps location calculator for two adjacent coordinates displayed in the map before.

# Transform

Following our insights and conclusions we applied transformations on the data before storage (which build upon the preprocessing transformations):

•	As the 'date' field, is a timestamp it is the most informative field and we consider it more trustworthy. The dateType(boolean is weekend feature) and currentHour seem like fields that can be relevant for future tasks, so we decide to recreate them based on the 'date'. We discard the others. 

•	Dropping the useless fetures (all-zero features), redundant features (features that have the same values) and inconsistent features (features that have uncertanties and that do not agree with each other)

•	Moreover, we drop 'angle' ,'systemTimestamp', 'direction' features as we think they are less relavent and we have many others features.

•	Changing ellapsedTime feature units to seconds by dividing by 1000. By looking on the data it seems that the values are measured as seconds, as the values are numbers that are multiplied by 1000.

•	After keeping only the relevant columns we drop duplicates rows. There are no null entries in the data

•	Splitting the table into 3 tables, according to the diagram and explanation we provide in the load section.

We are left with a more refined database that will allow us to perform exciting tasks in the next parts of the project.
We keep some features that seem interesting, but note that some of them still contain missing data or uncertainties or are highly unbalanced and we hope to address these issues in the future.

In [50]:
df = df.drop("calendar", "systemTimestamp", "probability", "poiId", "poiId2", "longitude", "latitude", "direction", "dateTypeEnum", "dateType", "currentHour", "angle", "filteredActualDelay", 'anomaly', 'gridID')
# display(df)

In [51]:
df = df.withColumn("is_weekend", date_format("date", 'EEE').isin(["Sat", "Sun"]).cast("int"))

In [52]:
df = df.withColumn("currentHour", F.hour("date"))

In [53]:
df = df.withColumn('ellapsedTime',df['ellapsedTime']/1000)

In [54]:
df = df.dropDuplicates()

In [55]:
display(df.select([F.count(F.when(F.isnan(c), c)).alias(c) for c, c_type in df.dtypes if c_type not in ['timestamp', 'boolean', 'array<double>']]))

actualDelay,areaId,areaId1,areaId2,areaId3,busStop,delay,distanceCovered,ellapsedTime,gridID,journeyPatternId,lineId,vehicleId,vehicleSpeed,is_weekend
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create tables from data

In [57]:
areaId_df = df.select('areaId', 'areaId1', 'areaId2', 'areaId3').distinct()
display(areaId_df)

areaId,areaId1,areaId2,areaId3
15002,14,234,3750
15158,14,236,3789
114,6,114,114
18307,17,285,4576
4621,17,288,4621
1011,15,252,1011
4255,16,265,4255
18275,17,285,4568
3896,14,243,3896
15537,14,242,3884


In [58]:
areaId_df = df.select('areaId', 'areaId1', 'areaId2', 'areaId3').distinct()

In [59]:
lineId_df = df.select('journeyPatternId', 'lineId').distinct()
display(lineId_df)

journeyPatternId,lineId
00130003,13
046A0007,46A
01451009,145
00111001,11
041X0003,41
017A1001,17A
00750002,75
016C1001,16
00260002,26
01021001,102


In [60]:
lineId_df = df.select('journeyPatternId', 'lineId').distinct()

In [61]:
main_df = df.drop('areaId1', 'areaId2', 'areaId3', 'lineId')
display(main_df)

actualDelay,anomaly,areaId,atStop,busStop,congestion,delay,distanceCovered,ellapsedTime,gridID,journeyPatternId,justLeftStop,justStopped,loc,vehicleId,vehicleSpeed,date,is_weekend
0,false,73080,true,494,false,-297,0.0,0.0,"159,101",00041001,false,false,"List(-6.251646, 53.341492)",44053,0,2017-07-03T10:56:40.000+0000,0
0,false,73095,false,407,false,185,0.0,0.0,"159,104",01511001,false,false,"List(-6.252083, 53.34835)",33404,0,2017-07-03T10:57:04.000+0000,0
20,false,15537,false,909,false,177,0.0,11.0,"159,95",00111001,false,false,"List(-6.252372, 53.331494)",44119,0,2017-07-03T10:57:06.000+0000,0
21,false,2344,false,2172,false,50,0.03926749814301063,12.0,"111,88",00131001,false,false,"List(-6.39477, 53.319287)",43121,0,2017-07-03T10:57:13.000+0000,0
20,false,3790,false,2476,false,173,0.0,20.0,"150,88",00831001,false,false,"List(-6.279686, 53.318711)",43136,0,2017-07-03T10:57:37.000+0000,0
34,false,71650,false,1480,false,262,0.0,22.0,"151,103",00791001,false,false,"List(-6.27627, 53.345882)",44077,0,2017-07-03T10:57:46.000+0000,0
0,false,17136,false,2035,false,-68,0.11510528871800657,20.0,"196,75",046A0001,false,false,"List(-6.142574, 53.295758)",44227,0,2017-07-03T10:57:48.000+0000,0
-22,false,561,false,4545,false,96,0.310779808203797,20.0,"100,88",01510001,false,false,"List(-6.42947, 53.319473)",33408,0,2017-07-03T10:57:59.000+0000,0
0,false,18263,false,747,false,-43,0.0,0.0,"158,101",00110001,false,false,"List(-6.256243, 53.342576)",44110,0,2017-07-03T10:58:13.000+0000,0
0,false,1012,false,2964,false,147,0.0,0.0,"200,17",01450007,false,false,"List(-6.129542, 53.19176)",43098,0,2017-07-03T10:58:24.000+0000,0


In [62]:
main_df = df.drop('areaId1', 'areaId2', 'areaId3', 'lineId')

In [63]:
main_df.count()

Out[15]: 237005813

# Load

In this phase we design and create our data werehouse.

We store the transformed data on a VM, utilizing ElasticSearch as our data warehouse.

•	**Datawarehouse installation**

The VM we received includes a fresh Ubuntu 18.04 installation. 
We utilize the fact that we have sudo rights to install the necessary components of our datawarehouse.

First we install docker and docker compose so that we are able to run the datawarehouse on a container in an easy manner.

Then we install ELK Stack (ElasticSearch + Kibana, and Logstash for the behind-the-scene data collection pipeline)

Using the docker containers we install two services - ElasticSearch and Kibana (Version 7.9.3) on two containers (with images and settings provided by the course, where elasticSearch is open on port 9200 and Kibana on 5601). They are connected via a bridge network and have access to a local volume on the VM to store and vizualize the data, for Elastic and Kibana respectively.  

Additionally, the VM is connected on the same local network as the Databricks environment, thus allowing a faster and securer way to transfer the data.

•	**Datawarehouse selection** 

We chose Elasticsearch as our datawarehouse vendor.

We are aware of the alternatives (such as mongoDb, cassandra and traditional sql tables), however elasticSearch seems to be best choice for us.

The main reasons why we chose Elasticsearch is its scalibity and fast performance. 

o	Scalability - Being able to manage huge indexes (in the order of hundreds of Gigabytes or Petabytes).

o	Fast performance - By using inverted indices, Elasticsearch allows for quick searchs even on very large data sets.

Additionally, ElasticSearch is Schema free, meaning it does not require defining the field type (and more) before the indexing process. If an object is indexed later with a new property, it will automatically be added to the mapping definitions.

Lastly, the easy setup procedure and the handy search&visualization features are handy and useful for us. 

To conclude, we wanted a feature-reach, secure, reliable and fast solution. Fortunately, Elastic provides us with all this.

•	**Create tables from data**

We split the data into 3 different tables: AreaId, LineId and Main table.

As we saw in the analysis section, each journeyPattern has exacly one lineId and each areaId has exacly one areaId1,areaId2,areaId3.

Therefore, we drop the areaId1,areaId2,areaId3 columns from the Main table and connect areaId with areaId1,areaId2,areaId3 in the AreaId table where areaId is the key.

Similarly, we drop the lineId column from the Main table and connect journeyPattern with lineId in the LineId table where journeyPattern is the key.

Our designed data warehouse is described below, by the star diagram.
This design allows us to reduce 4 of the features in the main data frame. Considering the fact that we have ~230 million records, and the fact that the two additional tables consist of only a few hunderd records, this allows us to store roughly 1 Billion less values (230Million\*4). Thus resulting in a smaller database.   

•	**Loading the data**

To upload the data we use a Connector (Sink) to Elastic.

We do so by first setting up the connection, defining the storage settings and mappings and finally writing to the datawarehouse. 

We explicitly define the type of 'date' and 'loc' as 'date' and 'geo_point' respectively so that Elastic will recognize these types correctly. 
The others are implictly recognized and defined.

In [65]:
img = mpimg.imread("/dbfs/FileStore/shared_uploads/shubi@campus.technion.ac.il/relationships.png")
plt.figure(figsize = (15,15))
imgplot = plt.imshow(img, interpolation='bilinear')
plt.axis('off')
plt.show()

In [66]:
ES_HOST = '10.0.0.5' # VM Server 
es = Elasticsearch([{'host': ES_HOST}], timeout=600000)

In [67]:
elastic_settings_mappings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    },
    "mappings": {
        "properties": {
            "date" : { "type": "date"},
            "loc" : { "type": "geo_point" }
        }
    }
}

elastic_settings_mappings_basic = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "refresh_interval" : -1
    }
}

In [68]:
def write_to_elastic(df_to_upload, index_name: str, settings=elastic_settings_mappings):
  
    if not es.indices.exists(index_name): # and overwrite:
        #es.indices.delete(index=index_name)
        es.indices.create(index=index_name, ignore=400, body=settings)

        df_to_upload.write.format("org.elasticsearch.spark.sql")\
            .option("es.resource", index_name)\
            .option("es.nodes.wan.only","true")\
            .option("es.port","9200")\
            .option("es.nodes",ES_HOST)\
            .option("es.nodes.client.only", "false")\
            .save()
    else:
        print("Index already exists")

In [69]:
write_to_elastic(df_to_upload=main_df, index_name="dublin-buses-main-df", settings=elastic_settings_mappings)

In [70]:
write_to_elastic(df_to_upload=areaId_df, index_name="dublin-buses-area-id", settings=elastic_settings_mappings_basic) 

In [71]:
write_to_elastic(df_to_upload=lineId_df, index_name="dublin-buses-journey-pattern", settings=elastic_settings_mappings_basic) 

In [72]:
write_to_elastic(df_to_upload=df, index_name="dublin-buses-sample", settings=elastic_settings_mappings)